In [2]:
!pip install osmnx
!pip install smart-mobility-utilities

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 3.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement smart-mobility-utilities (from versions: none)
ERROR: No matching distribution found for smart-mobility-utilities


In [ ]:
import osmnx
from smart_mobility_utilities.common import Node, cost
from smart_mobility_utilities.viz import draw_route
import random
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
reference = (43.661667, -79.395)
G = osmnx.graph_from_point(reference, dist=300, clean_periphery = True, simplify=True)
origin = Node(graph=G, osmid=55808290)
destination = Node(graph=G, osmid=389677909)
highlighted = [389677909, 55808290]
nc = ['red' if node in highlighted else '#336699' for node in G.nodes()]
ns = [50 if node in highlighted else 8 for node in G.nodes()]
fig, ax = osmnx.plot_graph(G, node_size=ns, node_color=nc, node_zorder=2)
alpha = 2
beta = 2
n = 500
Q = 1
pheremone_concentrations = dict()
known_routes = dict()
pheremone_concentrations = {(u,v):random.uniform(0,0.5) for [u,v] in G.edges()}
def pheremone(level, distance, alpha, beta):
    return level ** alpha * ((1/distance)) ** beta
for ant in tqdm(range(n)):
    frontier = [origin]
    explored = set()
    route = []
    found = False
    while frontier and not found:
        parent = frontier.pop(0)
        explored.add(parent)
        children = []
        children_pheremones = []
        for child in parent.expand():
            if child == destination:
                found = True
                route = child.path()
                continue
            if child not in explored:
                children.append(child)
                children_pheremones.append(
                    pheremone(
                        pheremone_concentrations[(parent.osmid, child.osmid)],
                        child.distance,
                        alpha,
                        beta,
                    )
                )
        if len(children) == 0:
            continue
        transition_probability = [
            children_pheremones[i] / sum(children_pheremones)
            for i in range(len(children_pheremones))
        ]
        chosen = random.choices(children, weights=transition_probability, k=1)[0]
        children.pop(children.index(chosen))
        frontier.extend(children)
        frontier.insert(0, chosen)
    for u, v in zip(route[:-1], route[1:]):
        length_of_edge = G[u][v][0]['length']
        pheremone_concentrations[(u,v)] += Q/length_of_edge
    route = tuple(route)
    if route in known_routes:
        known_routes[route] += 1
    else:
        known_routes[route] = 1
best_route = max(known_routes, key= known_routes.get)
times_used = known_routes[best_route]
route = list(best_route)
print("Cost:", cost(G,route))
print("Times used:",times_used)
draw_route(G,route)
costs = [cost(G,r) for r in known_routes]
used = list(known_routes.values())
costs, used = zip(*sorted(zip(costs,used)))
plt.plot(costs, used)
plt.xlabel('Solution cost')
plt.ylabel('Times travelled')
plt.show()